In [ ]:
import torch
import timm

In [ ]:
# Load model architecture
model = timm.create_model("mobilevitv2_100", pretrained=False, num_classes=9)  # Change num_classes if needed tf_efficientnetv2_s
model.load_state_dict(torch.load("/Users/nelson/py/paper_impl/emotion_classifier/models/best_model_ema_47.pth", map_location="cpu"))
model.eval()

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),  # Match MobileViTv2 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])


In [ ]:
import mediapipe as mp
import cv2

mp_face_detection = mp.solutions.face_detection
face_detector = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)


In [ ]:
# Emotion labels (example for 9 classes — change based on your dataset)
emotion_labels = ['neutral', 'happy', 'sad', 'angry', 'fearful', 'disgusted', 'surprised', 'confused', 'calm']

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detector.process(img_rgb)

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x1 = int(bboxC.xmin * iw)
            y1 = int(bboxC.ymin * ih)
            w = int(bboxC.width * iw)
            h = int(bboxC.height * ih)

            # Ensure box is within image bounds
            x1, y1 = max(x1, 0), max(y1, 0)
            x2, y2 = min(x1 + w, iw), min(y1 + h, ih)

            # Crop and preprocess face
            face = frame[y1:y2, x1:x2]
            if face.size == 0:
                continue

            face_tensor = transform(face).unsqueeze(0).to(device)

            # Run emotion prediction
            with torch.no_grad():
                output = model(face_tensor)
                pred = torch.argmax(output, dim=1).item()
                label = emotion_labels[pred]

            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow("Facial Emotion Recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
